# Kaggle Project: Identifying Age-Related Conditions

## Model: TensorFlow Decision Forest

### Libraries and Data

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold, train_test_split

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.model_selection import cross_val_score
import tensorflow_decision_forests as tfdf

## disable warnings from tfds
import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
train  = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/train.csv",  index_col="Id")
test   = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/test.csv",   index_col="Id")
greeks = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/greeks.csv", index_col="Id")

### Basic EDA

In [3]:
print(f'The training dataset has {train.shape[0]} rows and {train.shape[1]} columns')
print(f'The test dataset has {test.shape[0]} rows and {test.shape[1]} columns')

The training dataset has 617 rows and 57 columns
The test dataset has 5 rows and 56 columns


In [4]:
train.head(n = 10)

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
Id,,,,,,,,,,,,,,,,,,,,,
000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1
04517a3c90bd,0.209377,2615.81430,85.200147,8.541526,8.138688,4.013127,0.025578,12.547282,1.229900,5237.54088,...,0.173229,0.49706,1.164956,21.915512,72.611063,24177.595550,28.525186,82.527764,21.978000,0
049232ca8356,0.348249,1733.65412,85.200147,8.377385,15.312480,1.913544,0.025578,6.547778,1.229900,5710.46099,...,4.408484,0.86130,0.467337,17.878444,192.453107,3332.467494,34.166222,100.086808,0.065096,0
057287f2da6d,0.269199,966.45483,85.200147,21.174189,8.138688,4.987617,0.025578,9.408886,1.229900,5040.77914,...,6.591896,0.49706,0.277693,18.445866,109.693986,21371.759850,35.208102,31.424696,0.092873,0
0594b00fb30a,0.346113,3238.43674,85.200147,28.888816,8.138688,4.021986,0.025578,8.243016,3.626448,6569.37001,...,4.762291,1.18262,0.067730,17.245908,147.218610,4589.611956,29.771721,54.675576,0.073416,0


In [5]:
greeks.head(n = 10)

,Alpha,Beta,Gamma,Delta,Epsilon
Id,,,,,
000ff2bfdfe9,B,C,G,D,3/19/2019
007255e47698,A,C,M,B,Unknown
013f2bd269f5,A,C,M,B,Unknown
043ac50845d5,A,C,M,B,Unknown
044fb8a146ec,D,B,F,B,3/25/2020
04517a3c90bd,A,C,M,B,10/1/2019
049232ca8356,A,C,M,B,5/29/2019
057287f2da6d,A,C,M,B,4/24/2019
0594b00fb30a,A,C,M,B,2/18/2019


### Preprocessing

Merge greeks with train and test datasets:

In [6]:
train_greeks = train.merge(greeks, how='left', on='Id')
test_greeks  = test.merge(greeks, how='left', on='Id')

print(f'The training + greeks dataset has {train_greeks.shape[0]} rows and {test_greeks.shape[1]} columns')
print(f'The test + greeks dataset has {test_greeks.shape[0]} rows and {test_greeks.shape[1]} columns')

The training + greeks dataset has 617 rows and 61 columns
The test + greeks dataset has 5 rows and 61 columns


Create holdout set to estimate out of sample performance:

In [7]:
train_split, holdout_split = train_test_split(train, test_size = 0.20, random_state = 55)

Separate inputs and labels:

In [8]:
train_inputs = train_split.drop(columns=['Class'])
train_labels = train_split.Class

holdout_inputs = holdout_split.drop(columns=['Class'])
holdout_labels = holdout_split.Class

test_inputs = test

Define and apply preprocessor:

In [9]:
numeric_var     = train_inputs.select_dtypes(include=['float64', 'int64']).columns
categorical_var = train_inputs.select_dtypes(include=['object']).columns

In [10]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_var),
        ('cat', categorical_transformer, categorical_var)
    ])

In [11]:
## fit on training data
preprocessor.fit(train_inputs)

## transform
temp_train_inputs_processed   = preprocessor.transform(train_inputs)
temp_holdout_inputs_processed = preprocessor.transform(holdout_inputs)
temp_test_inputs_processed    = preprocessor.transform(test_inputs)

## convert back to DataFrame
train_inputs_processed   = pd.DataFrame(temp_train_inputs_processed, index = train_inputs.index, columns = preprocessor.get_feature_names_out())
holdout_inputs_processed = pd.DataFrame(temp_holdout_inputs_processed, index = holdout_inputs.index, columns = preprocessor.get_feature_names_out())
test_inputs_processed    = pd.DataFrame(temp_test_inputs_processed, index = test_inputs.index, columns = preprocessor.get_feature_names_out())

## concatenate labels
train_final   = train_inputs_processed.assign(Class = train_labels)
holdout_final = holdout_inputs_processed.assign(Class = holdout_labels)
test_final    = test_inputs_processed

### Modeling

Class Weighting:

In [12]:
# Calculate the number of samples for each label.
neg, pos = np.bincount(train_labels)

# Calculate total samples.
total = neg + pos

# Calculate the weight for each label.
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 0.60
Weight for class 1: 2.90


Fit and evaluate model using 5-Fold CV:

In [13]:
## initialize kfold object
kfold = KFold(n_splits=5)

## create empty lists to store outputs
accuracy = []
cross_entropy = []
models = []

for i, (train_index, valid_index) in enumerate(kfold.split(train_inputs)):

    ## display current iteration
    print(f'### Fold {i+1}')
    
    ## define temporary training and validation sets
    train_temp = train_final.iloc[train_index]
    valid_temp = train_final.iloc[valid_index]

    ## convert to tensorflow dataset
    train_tfds = tfdf.keras.pd_dataframe_to_tf_dataset(train_temp, label = 'Class')
    valid_tfds = tfdf.keras.pd_dataframe_to_tf_dataset(valid_temp, label = 'Class')

    ## initialize model object and define evaluation metrics
    rf = tfdf.keras.RandomForestModel()
    rf.compile(metrics = ['accuracy', 'binary_crossentropy'])

    ## fit and store model
    rf.fit(x = train_tfds, class_weight = class_weight)
    models.append(rf)

    ## make predictions on validation set and log accuracy
    evaluation = rf.evaluate(x = valid_tfds, return_dict=True)
    accuracy.append(evaluation["accuracy"])
    cross_entropy.append(evaluation["binary_crossentropy"])

## return metrics across folds
print(accuracy)
print(cross_entropy)

### Fold 1
Use /tmp/tmpldnkffvb as temporary training directory
Reading training dataset...
Training dataset read in 0:00:06.249367. Found 394 examples.
Training model...
Model trained in 0:00:00.182822
Compiling model...


[INFO 23-06-30 01:54:22.4110 UTC kernel.cc:1242] Loading model from path /tmp/tmpldnkffvb/model/ with prefix cf0b782a52604f34
[INFO 23-06-30 01:54:22.4340 UTC decision_forest.cc:660] Model loaded with 300 root(s), 10910 node(s), and 57 input feature(s).
[INFO 23-06-30 01:54:22.4341 UTC abstract_model.cc:1311] Engine "RandomForestOptPred" built
[INFO 23-06-30 01:54:22.4342 UTC kernel.cc:1074] Use fast generic engine


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.
1/1 [==============================] - 1s 604ms/step - loss: 0.0000e+00 - accuracy: 0.8586 - binary_crossentropy: 0.2932
### Fold 2
Use /tmp/tmpbq3w3344 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.860454. Found 394 examples.
Training model...
Model trained in 0:00:00.175430
Compiling model...


[INFO 23-06-30 01:54:26.7223 UTC kernel.cc:1242] Loading model from path /tmp/tmpbq3w3344/model/ with prefix c0002e32b10345f2
[INFO 23-06-30 01:54:26.7437 UTC decision_forest.cc:660] Model loaded with 300 root(s), 11616 node(s), and 57 input feature(s).
[INFO 23-06-30 01:54:26.7437 UTC kernel.cc:1074] Use fast generic engine


Model compiled.
1/1 [==============================] - 0s 294ms/step - loss: 0.0000e+00 - accuracy: 0.9798 - binary_crossentropy: 0.2312
### Fold 3
Use /tmp/tmp2rgwhmui as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.805510. Found 394 examples.
Training model...
Model trained in 0:00:00.134958
Compiling model...


...

Model compiled.
1/1 [==============================] - 0s 303ms/step - loss: 0.0000e+00 - accuracy: 0.8990 - binary_crossentropy: 0.2963
### Fold 4
Use /tmp/tmpnusowtp2 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.784484. Found 395 examples.
Training model...
Model trained in 0:00:00.141790
Compiling model...


[INFO 23-06-30 01:54:30.4171 UTC kernel.cc:1242] Loading model from path /tmp/tmpnusowtp2/model/ with prefix 9d381cc42ddc469d
[INFO 23-06-30 01:54:30.4366 UTC decision_forest.cc:660] Model loaded with 300 root(s), 11262 node(s), and 57 input feature(s).
[INFO 23-06-30 01:54:30.4366 UTC kernel.cc:1074] Use fast generic engine


Model compiled.
1/1 [==============================] - 0s 306ms/step - loss: 0.0000e+00 - accuracy: 0.9184 - binary_crossentropy: 0.2501
### Fold 5
Use /tmp/tmpn66inhcn as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.796139. Found 395 examples.
Training model...
Model trained in 0:00:00.136212
Compiling model...


[INFO 23-06-30 01:54:32.2709 UTC kernel.cc:1242] Loading model from path /tmp/tmpn66inhcn/model/ with prefix b7ea3a5f90b54c80
[INFO 23-06-30 01:54:32.2900 UTC decision_forest.cc:660] Model loaded with 300 root(s), 11604 node(s), and 57 input feature(s).
[INFO 23-06-30 01:54:32.2900 UTC kernel.cc:1074] Use fast generic engine


Model compiled.
1/1 [==============================] - 0s 280ms/step - loss: 0.0000e+00 - accuracy: 0.9388 - binary_crossentropy: 0.2298
[0.8585858345031738, 0.9797979593276978, 0.8989899158477783, 0.918367326259613, 0.9387755393981934]
[0.2931727468967438, 0.2312374860048294, 0.2963302731513977, 0.25013789534568787, 0.2298472821712494]


Holdout Set Evaluation:

### Predictions and Submission File

In [14]:
test_pred = models[2].predict(tfdf.keras.pd_dataframe_to_tf_dataset(test_final)).flatten()

submission = pd.DataFrame({
    'Id': test_inputs.index, 
    'class_0': 1 - test_pred, 
    'class_1': test_pred
})

submission.to_csv("submission.csv", index=False)

1/1 [==============================] - 0s 99ms/step
